In [1]:
# coding utf-8
"""
Created on Jul 31.2019
机器学习实战之决策树源码+我的一些注释
@author:Tobin
"""

'\nCreated on Jul 31.2019\n机器学习实战之决策树源码\n@author:Tobin\n'

In [2]:
# 计算给定数据集的香农熵，熵定义为信息的期望值。即要写一个计算给定数据集信息量的代码。我们使用频率近似概率。
# 数据集的形式是前n项为特征，最后一项为label值。使用一个字典记录下每个特征下数据的数量
from math import log
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel]+=1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob*log(prob, 2)
    return shannonEnt



In [3]:
# 创建数据集，使用calcShannonEnt函数，验证是否写正确
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [40]:
data, labels = createDataSet()
calcShannonEnt(data)

0.9709505944546686

In [8]:
# 写完了计算信息量的函数，需要一个按照给定特征进行数据集分割的函数，然后我们计算分割后的信息量，
# 再判断选择哪个特征划分数据集
# axis表示特征，value表示特征值，则相当于将某特征符合某值的数据分割出来
# 注意extend和append的区别，append将list作为元素插入，extend将两个list合并为一个list
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            # 这里去除了当前特征
            reducedFeatVec = featVec[:axis]    
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet


In [9]:
# 测试splitDataSet函数是否正确
splitDataSet(data, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [10]:
# 现在我们需要遍历所有特征，计算每个特征划分后的信息增益，返回信息增益最大的特征
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1     
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    # 遍历所有特征
    for i in range(numFeatures):     
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)    
        newEntropy = 0.0
        # 遍历该特征的所有特征值。
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        # 计算信息增益：熵 - 条件熵
        infoGain = baseEntropy - newEntropy     
        # 记录最大的信息熵， 和最大信息熵时候的特征。
        if (infoGain > bestInfoGain):       
            bestInfoGain = infoGain        
            bestFeature = i
    return bestFeature                 

In [11]:
# 测试chooseBestFeatureToSplit函数，使用自己构造的数据代码
chooseBestFeatureToSplit(data)

0

In [23]:
# 现在已经有了计算信息量的函数，划分数据集的函数，和选取最好的划分特征的函数
# 当我们划分完一次后，我们需要对剩下的各个分支继续进行这样的划分，直到满足递归条件
# 递归结束的条件是程序遍历完数据集的属性，或者每个分支下的所有实例都具有相同的分类
# 如果我们已经遍历完所有的属性，子结点中的数据还不是同属于同一类怎么办呢？此时使用多数表决的方式决定该子节点的分类。
# 统计个类别的数量， 进行排序， 返回数量最多的类别。


import operator
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        classCount[vote] += 1
    # 因为python3中dict已经没有iteritems这个属性，直接改为items即可
    # operator模块提供的itemgetter函数用于获取对象的哪些维的数据，参数为一些序号（即需要获取的数据在对象中的序号），下面看例子。
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    print(sortedClassCount)
    return sortedClassCount[0][0]

In [24]:
# classList = [1,2,3,4,3,2,1,1,1,4]
# majorityCnt(classList)

[(1, 4), (2, 2), (3, 2), (4, 2)]


1

In [32]:
# 我们已经有了所有的子模块，下面就可以递归创建决策树了，现在我们的算法是ID3算法，对于数值型数据比较乏力
# 树的结构只用字典进行存储
# 递归创建决策树
def createTree(dataSet,labels):
    # 获取所有样本的类别
    classList = [example[-1] for example in dataSet]
    # 两个递归边界，所有样本属于同一类或者特征遍历完毕，ID3每次删去一个特征，最终只剩下一个特征
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#stop splitting when all of the classes are equal
    # 如果特征已为空， 将类别数量最多的作为该节点的类别。
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    # 选取最好的特征作为此时的节点。
    bestFeat = chooseBestFeatureToSplit(dataSet)
    # 通过索引获取到最好的特征标识，用于下面字典记录书的节点。
    bestFeatLabel = labels[bestFeat]
    # 决策树
    myTree = {bestFeatLabel:{}}
    # 将最好的特征删除掉， 该特征后边已经用不到了， 将递归的重新选取除此之外的下一个特征，最为树的节点。
    del(labels[bestFeat])
    # 基于最好的特征继续构建树。
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    # 遍历最好特征的每一个特征值继续构建树。
    for value in uniqueVals:
        subLabels = labels[:]      
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree   

In [50]:
# 验证建树结果
data, labels = createDataSet()
myTree = createTree(data, labels)

In [45]:
# 决策树的一个好处时比较直观，书中还有可视化决策树的代码，但不是学习重点，在次略过
# 下面我们要使用构建好的决策树对测试数据进行预测，对照上面的验证结果更好理解，在递归地访问叶节点的过程中，featLabels和testVec是不变的
def classify(inputTree,featLabels,testVec):
    # 获取树的第一个keys
    # 由于python3改变了dict.keys,返回的是dict_keys对象,支持iterable 但不支持indexable，我们可以将其明确的转化成list
    firstStr = list(inputTree.keys())[0]
    # 通过keys获取values
    secondDict = inputTree[firstStr]
    # 通过keys， 获取特征列表中的特征的索引
    featIndex = featLabels.index(firstStr)
    # 通过特征索引获取测试数据中的特征值
    key = testVec[featIndex]
    # 根据上面获取到的特征值， 确定下一步往树的哪个分支走， 相当于又重新进入到下一个树， 递归上面的步骤。
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [51]:
# 分类数据测试
data, labels = createDataSet()
classify(myTree, labels, [1, 0])

'no'

In [57]:
# 现在我们拥有了训练好的决策树，我们要把它存储起来，就不用每次都训练一个决策树了
# python提供了pickle模块，可以序列化对象，序列化对象可以在磁盘上保存，并在需要的时候读取出来，任何对象都可以进行序列化操作。

def storeTree(inputTree,filename):
    import pickle
    # write() argument must be str, not bytes，修改w为wb，增加'rb'，默认为'r'。
    # 原因为：Python3给open函数添加了名为encoding的新参数，而这个新参数的默认值却是‘utf-8’。
    # 这样在文件句柄上进行read和write操作时，系统就要求开发者必须传入包含Unicode字符的实例，而不接受包含二进制数据的bytes实例。
    fw = open(filename,'wb')
    pickle.dump(inputTree,fw)
    fw.close()

# 加载树    
def grabTree(filename):
    import pickle
    fr = open(filename, 'rb')
    return pickle.load(fr)

In [59]:
# 测试存储与读取模块

myData, labels = createDataSet()
myTree = createTree(myData, labels)
print (myTree)
storeTree(myTree, 'myTree.pkl')


myTree = grabTree('myTree.pkl')
predict = classify(myTree, ['no surfacing','flippers'], [1,0])
print (predict)

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
no
